# Relatório Final - Análise de Séries Temporais de Nascimentos em Portugal
Este notebook apresenta uma análise compreensiva da série temporal de nascimentos em Portugal, abordando todos os aspectos requeridos no projeto.

## Preparação dos Dados
O dataset foi carregado e verificado quanto a dados omissos. As datas foram convertidas para o formato datetime e definidas como índice do DataFrame.

In [ ]:
import pandas as pd

# Load the dataset
file_path = '../data/processed/births_PT_limpo.csv'  # Update with the actual path
births_data = pd.read_csv(file_path)
births_data['Date'] = pd.to_datetime(births_data['Date'])
births_data.set_index('Date', inplace=True)
births_data.head()


## Análise Descritiva
A análise descritiva revela as características fundamentais dos dados. Foram geradas estatísticas descritivas e visualizações.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Summary statistics
display(births_data.describe())

# Histogram
plt.figure(figsize=(10, 6))
plt.hist(births_data['Births'], bins=30)
plt.title('Distribuição de Nascimentos')
plt.xlabel('Nascimentos')
plt.ylabel('Frequência')
plt.show()

# Box Plot
plt.figure(figsize=(10, 6))
sns.boxplot(births_data['Births'])
plt.title('Box Plot de Nascimentos')
plt.xlabel('Nascimentos')
plt.show()


## Visualização Temporal e Correlograma
A série temporal foi visualizada para identificar tendências, sazonalidade e outras características. Além disso, foi realizado um correlograma para analisar a autocorrelação.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

# Time series plot
plt.figure(figsize=(12, 6))
plt.plot(births_data['Births'], label='Births')
plt.title('Time Series of Births in Portugal')
plt.xlabel('Date')
plt.ylabel('Number of Births')
plt.legend()
plt.show()

# Correlogram
plot_acf(births_data['Births'].dropna(), lags=40)
plt.show()


## Análise de Estacionariedade
Foi realizada a análise de estacionariedade da série temporal para determinar se as propriedades estatísticas da série se mantêm constantes ao longo do tempo.

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Performing Augmented Dickey-Fuller test
adf_test = adfuller(births_data['Births'].dropna())

# Displaying ADF test results
adf_result = {
    'ADF Statistic': adf_test[0],
    'p-value': adf_test[1],
    'Used Lag': adf_test[2],
    'Number of Observations Used': adf_test[3],
    'Critical Values': adf_test[4],
}
adf_result


## Decomposição Temporal
Foram aplicados métodos de decomposição clássica e STL à série temporal para identificar e separar componentes como tendência, sazonalidade e resíduos.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose, STL

# Classical Decomposition
decompose_classical = seasonal_decompose(births_data['Births'], model='additive', period=12)
fig = decompose_classical.plot()
fig.set_size_inches(10, 8)
plt.show()

# STL Decomposition
stl = STL(births_data['Births'], seasonal=13)
result = stl.fit()
fig = result.plot()
fig.set_size_inches(10, 8)
plt.show()


## Conclusão
Este relatório forneceu uma análise detalhada da série temporal dos nascimentos em Portugal, abordando diversos aspectos como estacionariedade, sazonalidade e tendência. As conclusões derivadas desta análise são cruciais para a compreensão das dinâmicas temporais dos nascimentos no país.